In [1]:
import numpy as np
import pandas as pd
df=pd.read_csv('master_data_cleaned.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,Grabber,Price,year,style,miles,engine,power,transmission,fuel,owners,serv_hist,ULEZ,CatN,doors,model,issue_flag
0,NaN,550.0,2003.0,Saloon,137000.0,2.2,138.0,Manual,Diesel,3.0,NaN,0,0,4.0,Honda Accord,0
1,cheap and most reliable car,705.0,2003.0,Hatchback,106262.0,1.4,82.0,Manual,Petrol,3.0,NaN,0,1,5.0,Honda Jazz,0
2,honda civic 2002,950.0,2002.0,Hatchback,58000.0,1.4,89.0,Manual,Petrol,5.0,NaN,0,0,5.0,Honda Civic,0
3,july 2021 mot+2 keys+,995.0,2006.0,Hatchback,122500.0,1.4,82.0,Manual,Petrol,3.0,NaN,1,0,5.0,Honda Jazz,0
4,call 07977223024 for questions,995.0,2001.0,Hatchback,91762.0,NaN,89.0,Manual,Petrol,NaN,NaN,0,0,5.0,Honda Civic,0


In [2]:
missing_values=df.isnull().sum()
missing_values.sort_values(ascending=False,inplace=True)
print(missing_values)

serv_hist       87177
owners          58936
year            15253
Grabber         14384
miles           14032
fuel             8509
doors            7779
power            3303
engine           2857
style             970
transmission      111
issue_flag          0
model               0
CatN                0
ULEZ                0
Price               0
dtype: int64


In [3]:
#serv_hist likely to be useless as almost all should have it and theres no accurate way of filling. Also drop Grabber
df.drop(['serv_hist','Grabber'], axis=1, inplace=True)


In [4]:
#year and miles are also difficult to fill in, and note that:
df[df['year'].isnull() & df['miles'].isnull()].shape

(13827, 14)

In [5]:
#hence we drop these entries
df.dropna(axis=0, inplace=True, subset=['year'])
df.dropna(axis=0, inplace=True, subset=['miles'])

In [6]:
#fuel
def get_fuel(model): 
    try:
        return df[df['model']==model].fuel.mode()[0]
    except IndexError:
        return np.nan

df['fuel']=df['fuel'].fillna(df['model'].apply(get_fuel))
print(df['fuel'].isnull().sum())

1547


In [7]:
def get_doors(model):
    try:
        return df[df['model']==model].doors.mode()[0]
    except IndexError:
        return np.nan
df['doors']=df['doors'].fillna(df['model'].apply(get_doors))
print(df['doors'].isnull().sum())

#remove outliers where doors=0
df=df[df['doors']>0]

218


In [8]:
def get_power(model):
    try:
        return df[df['model']==model].power.median()
    except IndexError:
        return np.nan
    
df['power']=df['power'].fillna(df['model'].apply(get_power))
print(df['power'].isnull().sum())

#remove outliers where power=1
df=df[df['power']>1]

171


In [9]:
def get_engine(model):
    try:
        return df[df['model']==model].engine.mode()[0]
    except IndexError:
        return np.nan
    
df['engine']=df['engine'].fillna(df['model'].apply(get_engine))
print(df['engine'].isnull().sum())

159


In [10]:
def get_style(model):
    try:
        return df[df['model']==model]['style'].mode()[0]
    except IndexError:
        return np.nan

df['style']=df['style'].fillna(df['model'].apply(get_style))
print(df['style'].isnull().sum())

281


In [11]:
def get_transmission(model):
    try:
        return df[df['model']==model].transmission.mode()[0]
    except IndexError:
        return np.nan

df['transmission']=df['transmission'].fillna(df['model'].apply(get_transmission))
print(df['transmission'].isnull().sum())

0


In [12]:
#owners - this is likely going to be the most inaccurate imputation - but we will do mode by year. 
def get_owners(year):
    try:
        return df[df['year']==year].owners.mode()[0]
    except IndexError:
        return np.nan
    
df['owners']=df['owners'].fillna(df['year'].apply(get_owners))
print(df['owners'].isnull().sum())

2


In [13]:
for col in ['owners','fuel','doors','power','engine','style','transmission']:
    df.dropna(axis=0, inplace=True, subset=[col])
df.shape

(83260, 14)

In [14]:
missing_values=df.isnull().sum()
missing_values.sort_values(ascending=False,inplace=True)
print(missing_values)

issue_flag      0
model           0
doors           0
CatN            0
ULEZ            0
owners          0
fuel            0
transmission    0
power           0
engine          0
miles           0
style           0
year            0
Price           0
dtype: int64


In [15]:
#remove model and keep make
def get_make(model):
    words=model.split(' ')
    return words[0]
df['make']=df['model'].apply(get_make)
df.drop(axis=1, columns=['model'],inplace=True)
print(list(df['make'].unique()))

['Honda', 'Ford', 'Audi', 'BMW', 'Jaguar', 'Land', 'Mercedes-Benz', 'Mercedes', 'Nissan', 'Porsche', 'Renault', 'SEAT', 'SKODA', 'Toyota', 'Vauxhall', 'Volkswagen', 'Volvo', 'Peugeot', 'Mitsubishi', 'Hyundai', 'Fiat', 'Citroen', 'Abarth', 'Fiat\\Abarth', 'Alfa', 'Aston', 'Bentley', 'Chevrolet', 'Chrysler', 'KIA', 'Mazda6', 'Mazda', 'Mazda3', 'Mazda5', 'Mazda2', 'MAZDA', 'MAZDA2', 'MAZDA6', 'MAZDA3', 'mazda6', 'MINI', 'Suzuki', 'Dacia', 'Daihatsu', 'Dodge', 'DS', 'Ferrari', 'Infiniti', 'Isuzu', 'Jeep', 'Lamborghini', 'Lexus', 'Maserati', 'McLaren', 'MG', 'Mg', 'Rolls-Royce', 'Rover', 'Saab', 'Smart', 'Ssangyong', 'Subaru']


In [16]:
#clean make
df=df[df['make']!='Fiat\\Abarth']
def make_fixer(make):
    if make=='Land':
        return 'Land Rover'
    elif make=='Alfa':
        return 'Alfa Romeo'
    elif make=='Aston':
        return 'Aston Martin'
    elif make=='Mg':
        return 'MG'
    elif make=='Mazda6' or make=='Mazda3' or make=='Mazda5' or make=='MAZDA' or make=='MAZDA6' or make=='MAZDA3' or make=='mazda6' or make=='Mazda2' or make=='MAZDA2':
        return 'Mazda'
    else: 
        return make
df['make']=df['make'].apply(make_fixer)

In [17]:
#obvious outliers
df.describe()

,Price,year,miles,engine,power,owners,ULEZ,CatN,doors,issue_flag
count,8.325900e+04,83259.000000,83259.000000,83259.000000,83259.000000,83259.000000,83259.000000,83259.000000,83259.000000,83259.000000
mean,1.701535e+04,2012.968844,51033.389075,3.154590,165.252411,2.085192,0.747535,0.027961,4.307486,0.014809
std,4.654279e+04,5.826556,42920.609926,43.358652,116.195988,1.371068,0.434429,0.164862,1.112890,0.120789
min,1.230000e+02,1960.000000,2.000000,0.100000,29.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,3.295000e+03,2009.000000,15398.500000,1.400000,99.000000,1.000000,0.000000,0.000000,3.000000,0.000000
50%,8.995000e+03,2015.000000,39828.000000,1.600000,130.000000,2.000000,1.000000,0.000000,5.000000,0.000000
75%,1.773000e+04,2017.000000,80062.500000,2.000000,181.000000,3.000000,1.000000,0.000000,5.000000,0.000000
max,9.999999e+06,2020.000000,900000.000000,1750.000000,1998.000000,17.000000,1.000000,1.000000,8.000000,1.000000


In [18]:
df=df[df['engine']<10]
df=df[df['power']<1000]
df=df[df['Price']<9000000]
df.describe()

,Price,year,miles,engine,power,owners,ULEZ,CatN,doors,issue_flag
count,8.318700e+04,83187.000000,83187.000000,83187.000000,83187.00000,83187.000000,83187.000000,83187.000000,83187.000000,83187.000000
mean,1.689268e+04,2012.965439,51063.104235,1.947630,165.20362,2.085987,0.747328,0.027985,4.308245,0.014822
std,3.114361e+04,5.827887,42926.336176,0.996607,116.04986,1.371325,0.434547,0.164931,1.112469,0.120841
min,1.230000e+02,1960.000000,2.000000,0.100000,29.00000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,3.290000e+03,2009.000000,15427.000000,1.400000,99.00000,1.000000,0.000000,0.000000,3.000000,0.000000
50%,8.995000e+03,2015.000000,39876.000000,1.600000,130.00000,2.000000,1.000000,0.000000,5.000000,0.000000
75%,1.771500e+04,2017.000000,80151.500000,2.000000,180.00000,3.000000,1.000000,0.000000,5.000000,0.000000
max,1.395000e+06,2020.000000,900000.000000,6.800000,800.00000,17.000000,1.000000,1.000000,8.000000,1.000000


In [19]:
df['year']=df['year'].astype('int')
df['owners']=df['owners'].astype('int')
df['doors']=df['doors'].astype('int')
df.head()

,Price,year,style,miles,engine,power,transmission,fuel,owners,ULEZ,CatN,doors,issue_flag,make
0,550.0,2003,Saloon,137000.0,2.2,138.0,Manual,Diesel,3,0,0,4,0,Honda
1,705.0,2003,Hatchback,106262.0,1.4,82.0,Manual,Petrol,3,0,1,5,0,Honda
2,950.0,2002,Hatchback,58000.0,1.4,89.0,Manual,Petrol,5,0,0,5,0,Honda
3,995.0,2006,Hatchback,122500.0,1.4,82.0,Manual,Petrol,3,1,0,5,0,Honda
4,995.0,2001,Hatchback,91762.0,1.6,89.0,Manual,Petrol,5,0,0,5,0,Honda


In [20]:
df.to_csv('master_final.csv')